In [1]:
using Pkg
Pkg.activate("/media/mat/HDD/AdaptiveTransportMap/")

 Activating environment at `/media/mat/HDD/AdaptiveTransportMap/Project.toml`


In [76]:
using Revise
using AdaptiveTransportMap
using LinearAlgebra
using Test
using ForwardDiff
using SpecialFunctions
using Polynomials
using BenchmarkTools
using TransportMap
using QuadGK
using FastGaussQuadrature
using AdaptiveTransportMap: derivative, vander, scale!, evaluate
using DiffResults
using Distributions
using Random
using LoopVectorization
using Optim
using Roots
using NLsolve
using SparseArrays

In [8]:
Nx = 2
Ne = 8
m = 5
ens = EnsembleState(Nx, Ne)

ens.S .=  [0.267333   1.43021;
          0.364979   0.607224;
         -1.23693    0.249277;
         -2.0526     0.915629;
         -0.182465   0.415874;
          0.412907   1.01672;
          1.41332   -0.918205;
          0.766647  -1.00445]';

X = ens.S 
idx = [0 0; 0 1; 1 0; 1 1]

B = MultiBasis(BasisProHermite(6; scaled =true), Nx)

coeff = [ -0.00319245982105743; 0.8024007165102715; -0.40077744068752075; -1.0563328221249926]

f = ParametricFunction(ExpandedFunction(B, idx, coeff))
R = IntegratedFunction(f);

# F = evaluate(R, X)

F = [  1.4856969015693695;
       0.5978925425658191;
       0.4593757680535011;
       1.1805056241296106;
       0.5264028580778785;
       1.0159025856390884;
      -0.9829981708131781;
      -1.098528178152973];

# Naive approach
Xd = zeros(Ne)
@time for i=1:Ne
    Xd[i] = find_zero(t->evaluate(R, reshape([X[1,i]; t],(2,1)))[1]-F[i], (-5.0, 5.0), Roots.Brent())
end

norm(Xd - X[2,:])<1e-5

  0.086258 seconds (205.47 k allocations: 9.818 MiB)


true

In [59]:
Ne = 500
X = randn(Nx, Ne)
@time F = evaluate(R, X);

  0.000781 seconds (699 allocations: 922.625 KiB)


In [50]:
ψdiag = repeated_evaluate_basis(R.f.f, zeros(Ne));
ψoff = evaluate_offdiagbasis(R.f, vcat(X[1:end-1,:], zeros(1, Ne)));
cache = zeros(Ne);

In [52]:
output = F - (ψdiag .* ψoff)*R.f.f.coeff;

In [75]:
@btime results = nlsolve(only_fj!(functional!($cache, $ψoff, $output, $R)), $zeros(Ne); method = :newton);

  4.507 ms (3330 allocations: 8.17 MiB)


In [81]:
spzeros(2,2)

2×2 SparseMatrixCSC{Float64,Int64} with 0 stored entries

In [62]:
Diagonal(randn(10))

10×10 Diagonal{Float64,Array{Float64,1}}:
 -1.60909    ⋅          ⋅         ⋅        …    ⋅         ⋅         ⋅ 
   ⋅       -0.103464    ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅        -0.883652   ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅        0.583092       ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅        …    ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅           -0.765556   ⋅         ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅        0.257673   ⋅ 
   ⋅         ⋅          ⋅         ⋅             ⋅         ⋅        0.974417

In [19]:
fieldnames(typeof(results))

(:method, :initial_x, :zero, :residual_norm, :iterations, :x_converged, :xtol, :f_converged, :ftol, :trace, :f_calls, :g_calls)

In [20]:
results.x_converged

false

In [72]:
function f!(F, x)
    F[1] = (x[1]+3)*(x[2]^3-7)+18
    F[2] = sin(x[2]*exp(x[1])-1)
end

function j!(J, x)
    J[1, 1] = x[2]^3-7
    J[1, 2] = 3*x[2]^2*(x[1]+3)
    u = exp(x[1])*cos(x[2]*exp(x[1])-1)
    J[2, 1] = x[2]*u
    J[2, 2] = u
end

nlsolve(f!, j!, [ 0.1; 1.2])

Results of Nonlinear Solver Algorithm
 * Algorithm: Trust-region with dogleg and autoscaling
 * Starting Point: [0.1, 1.2]
 * Zero: [-3.7818049096324184e-16, 1.0000000000000002]
 * Inf-norm of residuals: 0.000000
 * Iterations: 4
 * Convergence: true
   * |x - x'| < 0.0e+00: false
   * |f(x)| < 1.0e-08: true
 * Function Calls (f): 5
 * Jacobian Calls (df/dx): 5

In [68]:
?OnceDifferentiable

search: OnceDifferentiable



No documentation found.

`NLSolversBase.OnceDifferentiable` is of type `UnionAll`.

# Summary

```
struct UnionAll <: Type{T}
```

# Fields

```
var  :: TypeVar
body :: Any
```

# Supertype Hierarchy

```
UnionAll <: Type{T} <: Any
```


In [86]:
Ne = 200
function f_sparse!(F, x)
    @inbounds for (i, xi) in enumerate(x)
        F[i] = exp(-xi^2/(2*i))*erf(xi)
    end 
end

function g_sparse!(J, x)
    @inbounds for (i, xi) in enumerate(x)
        J[i,i] = (-xi/i)*exp(-xi^2/(2*i))*erf(xi) + exp(-xi^2/(2*i))*(2/√π)*exp(-xi^2)
    end 
end

df = OnceDifferentiable(f_sparse!, g_sparse!, randn(Ne), rand(Ne), spzeros(Ne, Ne))

F = rand(Ne);

In [97]:
@time r = nlsolve(df, F, method = :newton);

  0.000530 seconds (489 allocations: 298.641 KiB)
